In [1]:
import collections
import string
import itertools
import re

In [2]:
Link = collections.namedtuple('Link', 'height left right')

In [3]:
def extract_pairs(net_string):
    return [[int(pi) for pi in p.split(', ')] for p in net_string[1:-1].split('), (')]

In [4]:
def read_net_string(net_string):
    return [Link(h, l, r) for h, (l, r) in enumerate(extract_pairs(net_string))]

In [5]:
def read_net(filename):
    with open(filename) as f:
        pairs = [re.split('\D+', p.strip()) for p in f.readlines()]
        lrs = [(int(lr[1]), int(lr[2])) for lr in pairs]
        return [Link(h, l, r) 
                for h, (l, r) in enumerate(lrs)]

In [6]:
small_net = read_net('04-small.txt')
small_net

[Link(height=0, left=2, right=3),
 Link(height=1, left=2, right=6),
 Link(height=2, left=3, right=7),
 Link(height=3, left=5, right=6),
 Link(height=4, left=0, right=1),
 Link(height=5, left=0, right=1),
 Link(height=6, left=6, right=7),
 Link(height=7, left=2, right=5),
 Link(height=8, left=6, right=9),
 Link(height=9, left=4, right=8),
 Link(height=10, left=0, right=2),
 Link(height=11, left=5, right=7),
 Link(height=12, left=4, right=8),
 Link(height=13, left=1, right=5),
 Link(height=14, left=6, right=8),
 Link(height=15, left=6, right=9),
 Link(height=16, left=2, right=5),
 Link(height=17, left=1, right=8),
 Link(height=18, left=5, right=7),
 Link(height=19, left=2, right=9)]

In [7]:
net = read_net('04-lines.txt')
len(net)

10135

In [8]:
def show_net(links, pair_sep=', '):
    return pair_sep.join('({}, {})'.format(l.left, l.right) for l in sorted(links))

In [9]:
def link_ends(link):
    return set((link.left, link.right))

In [10]:
def follow(initial_line, links):
    line = initial_line
    heights = sorted(set(l.height for l in links))
    for h in heights:
        for l in [l for l in links if l.height == h]:
            if line in link_ends(l):
                line = [e for e in link_ends(l) if e != line][0]
#                 print(l, line)
    return line

In [11]:
def pack(net):
    packed_links = []
    line_heights = collections.defaultdict(lambda: -1)
    for link in sorted(net):
        link_height = max(line_heights[link.left], line_heights[link.right]) + 1
        line_heights[link.left] = link_height
        line_heights[link.right] = link_height
        packed_links += [Link(link_height, link.left, link.right)]
    return sorted(packed_links)

In [12]:
max(l.height for l in small_net)

19

In [13]:
max(l.height for l in pack(small_net))

7

In [14]:
max(l.height for l in net)

10134

In [15]:
pnet = pack(net)

In [16]:
max(l.height for l in pnet)

2286

In [17]:
def height_groups(net):
    return {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}

In [18]:
def follow_many(in_sequence, net):
    hgs = height_groups(net)
    seq = list(in_sequence)
    for h in hgs:
        for link in hgs[h]:
            seq[link.right], seq[link.left] = seq[link.left], seq[link.right]
    return seq

In [19]:
''.join(follow_many('abcdefghij', small_net))

'djihegcafb'

In [20]:
%%timeit
follow_many('abcdefghij', small_net)

10000 loops, best of 3: 50.4 µs per loop


In [21]:
''.join(follow_many(string.ascii_lowercase, net))

'doqzmbishkwunvltpcexyjgfra'

In [22]:
%%timeit
follow_many(string.ascii_lowercase, net)

10 loops, best of 3: 21 ms per loop
